<a href="https://colab.research.google.com/github/jessalva/Adversarial-Attacks-Final-Year-Project/blob/master/Dataset_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install captcha

    100% |████████████████████████████████| 102kB 2.7MB/s 


In [0]:
!pip install progressbar

  Stored in directory: /root/.cache/pip/wheels/c0/e9/6b/ea01090205e285175842339aa3b491adeb4015206cda272ff0
Successfully built progressbar


In [0]:
import argparse
import json
import string
import os
import shutil
import uuid
from captcha.image import ImageCaptcha
import random
import itertools
import progressbar

In [0]:
def get_choices():
    return tuple([i for i in string.ascii_lowercase])

In [0]:
from IPython.display import HTML, display
import time

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))


In [0]:
def _gen_captcha(img_dir, num_per_image, n, width, height, choices):
    if os.path.exists(img_dir):
        shutil.rmtree(img_dir)
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)

    image = ImageCaptcha(width=width, height=height)
    bar = progressbar.ProgressBar(maxval=24000000, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    print('generating %s epoches of captchas in %s' % (n, img_dir))
    k = 0
    bar.start()

    out = display(progress(0, 1000000), display_id=True)

    for i in itertools.combinations(choices, num_per_image):
         for j in itertools.permutations(i,6):                
                captcha = ''.join(j)
                fn = os.path.join(img_dir, '%s.png' % (captcha))
                image.write(captcha, fn)
                k += 1
                bar.update(k) 
                out.update(progress(k, 1000000))
                if k >= 1000000:
                    bar.finish()                    
                    break            
         if k >= 1000000:
                break

In [0]:
def build_file_path(x):
    return os.path.join(os.getcwd(), x)


In [0]:
def gen_dataset():
    n_epoch = 1
    num_per_image = 6

    choices = get_choices()

    width = 160
    height = 100

    # meta info
    meta = {
        'num_per_image': num_per_image,
        'label_size': len(choices),
        'label_choices': ''.join(choices),
        'n_epoch': n_epoch,
        'width': width,
        'height': height,
    }
    print('%s choices: %s' % (len(choices), ''.join(choices) or None))
    _gen_captcha(build_file_path('train'), num_per_image, n_epoch, width, height, choices=choices)



In [0]:
gen_dataset()

26 choices: abcdefghijklmnopqrstuvwxyz
generating 1 epoches of captchas in /content/train


In [0]:
%mkdir /content/test
%mkdir /content/validation
%cd /content/train

mkdir: cannot create directory ‘/content/test’: File exists
mkdir: cannot create directory ‘/content/validation’: File exists
/content/train


In [0]:
!ls | shuf -n 10000 | xargs -i cp {} ../test/
!ls | shuf -n 10000 | xargs -i cp {} ../validation

In [0]:
import glob
import re
import tensorflow as tf
from PIL import Image

char_dict = {
'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8,
'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17,
's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25
}

In [0]:
def load_image(filename):
  with tf.gfile.FastGFile(filename, 'rb') as f:
    image_data = f.read()
  label = re.search('\/(\w{6})\.png', filename).group(1)
  label_num = []
  for c in label:
    label_num.append(char_dict[c])
  return tf.compat.as_bytes(image_data), label_num

In [0]:
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_list_feature(list):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=list))

In [0]:
def write_to_trf_file(tfr_file_name, images_path):
  writer = tf.python_io.TFRecordWriter(tfr_file_name)
  images_list = glob.glob(images_path+"/*.png")
  for image_path in images_list:
    image, label = load_image(image_path)
    feature = {'label': _int64_list_feature(label),
               'image': _bytes_feature(image)}
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    writer.write(example.SerializeToString())

  writer.close()

In [0]:
%cd /
write_to_trf_file('./captcha_train.tfrecord', '/content/train')
write_to_trf_file('./captcha_validation.tfrecord', '/content/validation')
write_to_trf_file('./captcha_test.tfrecord', '/content/test')

/


In [0]:
!ls *.tfrecord

captcha_test.tfrecord  captcha_train.tfrecord  captcha_validation.tfrecord


In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 17.5MB/s 


In [0]:
upload = drive.CreateFile({'title': 'captcha_test.tfrecord'})
upload.SetContentFile('captcha_test.tfrecord')
upload.Upload()

upload = drive.CreateFile({'title': 'captcha_train.tfrecord'})
upload.SetContentFile('captcha_train.tfrecord')
upload.Upload()

upload = drive.CreateFile({'title': 'captcha_validation.tfrecord'})
upload.SetContentFile('captcha_validation.tfrecord')
upload.Upload()

